**Import Libraries and variables**

In [60]:
import pandas as pd
import re
import importlib
import vehicle_map_1,vehicle_map_2

importlib.reload(vehicle_map_1)
importlib.reload(vehicle_map_2)
from vehicle_map_1 import vehicle_map_1
from vehicle_map_2 import vehicle_map_2

df=pd.read_csv("./datasets/cleaned_vehicles.csv")

**Add,remove and short**

In [61]:
noise_words=["brand new","b/n","brand n","brandnew","new","used","1st","1st owner","petrol","diesel","hybrid","electric","Anniversary",
"second","2nd owner","company maintained","first","owner","owners","manual",
"full option","full options","full-option","full-options","full optioned","full-option","highest spec","high spec","full spec","Pre Order","Moon Roof",
"moonroof","(brandnew)","(brand new)","(moonroof)"]

#Lowercase Model
df["Model"]=df["Model"].str.lower().str.strip()
#Sort dataset
df=df.sort_values(by=["Manufacturer","Model","Year"],ascending=[True,True,True])


print(f"Saved {len(df)} records to before cleaned_vehicles.csv")
#----------removing noise words from Model column----------
for word in noise_words:
    df["Model"]=df["Model"].str.replace(word,"",regex=True)
df["Model"]=df["Model"].str.replace(r"\s+"," ",regex=True).str.strip()

#----------removing duplicates----------
df=df.drop_duplicates(subset=["Manufacturer","Model","Year"],keep="first")
df = df[df["Year"] >= 2005]

#----------Replace words in Model column----------
mask = (df["Manufacturer"].str.strip().str.lower() == "ford") & \
       (df["Model"].str.lower().str.contains("raptor ranger"))
df.loc[mask, "Model"] = df.loc[mask, "Model"].str.replace(
    r"raptor\s*ranger", "ranger raptor", regex=True, case=False
)
#----------Remove floating points----------
df["Seating Capacity"] = df["Seating Capacity"].astype("Int64")


desired_order = [
    "Manufacturer", "Model", "Year", "Fuel Type", "Body Type", "Seating Capacity",
    "Ground Clearance (range)", "Body Type 1 (sedan)", "Body Type 2 (hatchback)",
    "Body Type 3 (suv)", "Body Type 4 (mpv)", "Body Type 5 (pickup)", 
    "Body Type 6 (coupe)", "Body Type 7 (convertible)", "Body Type 8 (wagon)",
    "Body Type 9 (van)", "Body Type 10 (crossover)", "Body Type 11 (kei / microvan)",
    "Body Type 12 (roadster)", "Body Type 13 (other)", "Body Type 14 (liftback)",
    "Body Type 15 (mpv / minivan)", "City/Urban", "Suburban/Normal", 
    "Mid Off-Road", "Off-Road/Hilly Terrain", 
    "Average Price (0-20k km)", "Average Price (20k-50k km)",
    "Average Price (50k-100k km)", "Average Price (100k+ km)"
]

columns_to_use = [col for col in desired_order if col in df.columns]
df = df[columns_to_use]


Saved 1982 records to before cleaned_vehicles.csv


In [62]:
vehicle_maps={**vehicle_map_1,**vehicle_map_2}

for brand, attributes in vehicle_maps.items():
    for model, seating_capacity in attributes["seating_capacity"].items():
        df.loc[
            (df["Manufacturer"].str.strip().str.lower() == brand.strip().lower())
            & (df["Model"].str.contains(model, case=False, regex=False)),
            "Seating Capacity",
        ] = seating_capacity

    for model, btype in attributes["body_type"].items():
        mask = (df["Manufacturer"] == brand) & (df["Model"].str.lower().eq(model))
        df.loc[mask, "Body Type"] = btype
        mask = (df["Manufacturer"] == brand) & (
            df["Model"].str.lower().str.contains(model)
        )
        df.loc[mask & df["Body Type"].isna(), "Body Type"] = btype

    print(f"Total {brand} cars: {df[df['Manufacturer'] == brand].shape[0]}")

Total Audi cars: 81
Total BMW cars: 129
Total BYD cars: 6
Total Chery cars: 5
Total Chevrolet cars: 0
Total DFSK cars: 9
Total Daihatsu cars: 63
Total Datsun cars: 2
Total Dongfeng cars: 1
Total Fiat cars: 1
Total Ford cars: 33
Total Honda cars: 165
Total Hyundai cars: 30
Total Isuzu cars: 0
Total JAC cars: 1
Total Jaecoo cars: 3
Total JMEV cars: 1
Total Jaguar cars: 5
Total Jeep cars: 8
Total Kia cars: 60
Total Land Rover cars: 107
Total Lexus cars: 14
Total MG cars: 8
Total Mahindra cars: 12
Total Maruti Suzuki cars: 47
Total Mazda cars: 20
Total Mercedes Benz cars: 119
Total Micro cars: 39
Total Mitsubishi cars: 91
Total Nissan cars: 114
Total Perodua cars: 14
Total Peugeot cars: 18
Total Porsche cars: 6
Total Renault cars: 6
Total SsangYong  cars: 0
Total Subaru cars: 3
Total Suzuki cars: 156
Total Tata cars: 20
Total Tesla cars: 5
Total Toyota cars: 557
Total Volkswagen cars: 13
Total Zotye cars: 3


In [63]:
df["Body Type"] = df["Body Type"].str.lower()

body_type_map = {
    "sedan": "Body Type 1 (sedan)",
    "hatchback": "Body Type 2 (hatchback)",
    "suv": "Body Type 3 (suv)",
    "mpv": "Body Type 4 (mpv)",
    "pickup": "Body Type 5 (pickup)",
    "coupe": "Body Type 6 (coupe)",
    "convertible": "Body Type 7 (convertible)",
    "wagon": "Body Type 8 (wagon)",
    "van": "Body Type 9 (van)",
    "crossover": "Body Type 10 (crossover)",
    "kei / microvan": "Body Type 11 (kei / microvan)",
    "roadster": "Body Type 12 (roadster)",
    "other": "Body Type 13 (other)",
    "liftback": "Body Type 14 (liftback)",
    "mpv / minivan": "Body Type 15 (mpv / minivan)",

}

for col_name in body_type_map.values():
    df[col_name] = 0

for btype, col_name in body_type_map.items():
    df.loc[df["Body Type"].str.contains(btype, na=False), col_name] = 1

In [64]:
df["Ground Clearance (range)"] = (
    df["Ground Clearance (range)"]
    .astype(str)
    .str.extract(r'(\d+\.?\d*)')[0]
    .astype(float)
    .astype("Int64") 
)

gc = df["Ground Clearance (range)"].fillna(0)

df.loc[gc <= 130, "City/Urban"] = 1 # City/Urban: <= 130
df.loc[(gc > 130) & (gc <= 170), ["City/Urban", "Suburban/Normal"]] = 1 # Suburban/Normal: 131–170
df.loc[(gc > 170) & (gc <= 200), ["City/Urban","Suburban/Normal","Mid Off-Road"]] = 1 # Mid Off-Road: 171–200
df.loc[gc > 200, ["City/Urban","Suburban/Normal","Mid Off-Road","Off-Road/Hilly Terrain"]] = 1 # Off-Road/Hilly Terrain: >200


In [65]:
df.to_csv("./datasets/cleaned_vehicles.csv",index=False)
print(f"Saved {len(df)} records to cleaned_vehicles.csv")

Saved 1982 records to cleaned_vehicles.csv


**Merge Final Year Filtered Dataset**

In [66]:
group_cols = [c for c in df.columns if c != "Year"]

# Custom function to merge years
def merge_years(years):
    years = sorted(set(years))
    if len(years) == 1:
        return str(years[0])
    # Check if continuous
    if years == list(range(min(years), max(years) + 1)):
        return f"{min(years)}–{max(years)}"
    else:
        return ",".join(map(str, years))

# Apply grouping + year merge
df_merged = df.groupby(group_cols, as_index=False).agg({"Year": merge_years})

# Save back to CSV
df_merged.to_csv("./datasets/vehicles_merged.csv", index=False)
print(f"Saved {len(df_merged)} records to vehicles_merged.csv")

Saved 1158 records to vehicles_merged.csv
